# Python 일급 함수(객체) - 기본 특징
* 런타임 초기화
* 변수 할당 가능
* 함수 인수 전달 가능
* 함수 결과 반환 가능(return)

In [15]:
# 함수 객체

def factorial(n):
    '''Factorial Function -> n : int'''
    if n == 1: # n <2
        return 1
    return n*factorial(n-1)

class A:
    pass

print(factorial(5)) # 이런식으로 재귀함수를 통하여 팩토리얼을 구현할 수 있다,
print()
print(factorial.__doc__)
print()
print(type(factorial), type(A)) 
print()
print(set(sorted(dir(factorial))) - set(sorted(dir(A)))) # 뭐.. 이렇게 차이가 있구먼~ 정도로
print()
print(factorial.__name__)
print()
print(factorial.__code__) # 이런식으로 속성 값들을 들여다 볼 수 있다.

# => 즉, 함수는 객체 취급을 하고 있다.

120

Factorial Function -> n : int

<class 'function'> <class 'type'>

{'__closure__', '__kwdefaults__', '__name__', '__globals__', '__qualname__', '__defaults__', '__annotations__', '__get__', '__code__', '__call__'}

factorial

<code object factorial at 0x000001295B41BB30, file "<ipython-input-15-5d37f1e17c2b>", line 3>


In [19]:
var_func = factorial # 함수 자체를 변수로 할당하였다.

print(var_func)
print()
print(var_func(10))
print()
print(map(var_func, range(1,11)))
print()
print(list(map(var_func, range(1,11))))

<function factorial at 0x000001295E51C550>

3628800


[1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


## 함수 인수 전달 및 함수로 결과 반환 -> 고위 함수(Higher-order function)
* map, filter, reduce 등

In [20]:
print(list(map(var_func, filter(lambda x: x % 2, range(1,6)))))
print([var_func(i) for i in range(1,6) if i % 2])

[1, 6, 120]
[1, 6, 120]


### reduce()

In [23]:
from functools import reduce
from operator import add

print(reduce(add, range(1,11))) # 감소하면서 누적된다.
print(sum(range(1,11)))

55
55


 ### 익명함수(lambda)
* 가급적 주석 작성
* 가급적 함수 사용
* 일반 함수 형태로 리팩토링 권장

In [24]:
print(reduce(lambda x, t: x + t, range(1,11)))

55


### Callable : 호출 연산자 -> 메소드 형태로 호출 가능한지 확인
* 호출 가능 확인

In [30]:
print(callable(str), callable(list), callable(var_func), callable(3.14))

True True True False


### partial 사용법 : 인수 고정 -> 콜백 함수에 사용

In [29]:
from operator import mul
from functools import partial

print(mul(10,10))

# 인수 고정
five = partial(mul, 5)

# 고정 추가
six = partial(five, 6)

print(five(10))
print(six())
print([five(i) for i in range(1,11)])
print(list(map(five, range(1,11))))

100
50
30
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
[5, 10, 15, 20, 25, 30, 35, 40, 45, 50]


# 일급함수- 클로저

In [31]:
# Ex1

def func_v1(a):
    print(a)
    print(b)
    
func_v1(10)

10


NameError: name 'b' is not defined

In [32]:
#Ex2

b = 20 # 글로벌 

def func_v2(a): # 로컬
    print(a)
    print(b)
    
func_v2(10)

10
20


In [33]:
#Ex3

c = 30

def func_v3(a): 
    print(a)
    print(c)
    c = 40
    
func_v3(10)

10


UnboundLocalError: local variable 'c' referenced before assignment

In [35]:
#Ex3

c = 30

def func_v3(a):
    c = 40
    print(a)
    print(c)
    

print('>>', c)    
func_v3(10)

>> 30
10
40


In [37]:
#Ex3

c = 30

def func_v3(a):
    global c
    print(a)
    print(c)
    c = 40

print('>>', c)    
func_v3(10)
print(">>>", c)

>> 30
10
30
>>> 40


# Closure(클로저) 사용 이유
* 서버 프로그래밍 -> 동시성(Concurrency)제어 -> 메모리 공간에 여러 자원이 접근 -> 교착상태(Dead Lock)
* 메모리를 공유하지 않고 메시지 전달로 처리하기 위한 -> Erlang
* 클로저는 공유하되 변경되지 않는(Immutable, Read Only) 적극적으로 사용 -> 함수형 프로그래밍
* 클로저는 불변자료구조 및 atom, STM -> 멀티스레드(Coroutine) 프로그래밍에 강점

In [38]:
a = 100

print(a + 100)
print(a + 1000)

# 결과 누적(함수 사용)
print(sum(range(1,51)))
print(sum(range(51,101)))

print()
print()

# 클래스 이용
class Averager():
    def __init__(self):
        self._series = []

    def __call__(self, v):
        self._series.append(v)
        print('inner >>> {} / {}'.format(self._series, len(self._series)))
        return sum(self._series) / len(self._series)


# 인스턴스 생성
averager_cls = Averager()

# 누적
print(averager_cls(15))
print(averager_cls(35))
print(averager_cls(40))


200
1100
1275
3775


inner >>> [15] / 1
15.0
inner >>> [15, 35] / 2
25.0
inner >>> [15, 35, 40] / 3
30.0


In [3]:
# Closure 사용

def closure_ex1():
    # Free variable
    # 클로저 영역
    series = []
    def averager(v):
        series.append(v)
        print('onner >>> {}/{}'.format(series, len(series)))
        return sum(series)/ len(series)
    return averager

avg_closure1 = closure_ex1()

print(avg_closure1(10))
print(avg_closure1(30))
print(avg_closure1(50))

onner >>> [10]/1
10.0
onner >>> [10, 30]/2
20.0
onner >>> [10, 30, 50]/3
30.0


In [4]:
# function inspection
print(dir(avg_closure1))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [5]:
print(dir(avg_closure1.__code__))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'co_argcount', 'co_cellvars', 'co_code', 'co_consts', 'co_filename', 'co_firstlineno', 'co_flags', 'co_freevars', 'co_kwonlyargcount', 'co_lnotab', 'co_name', 'co_names', 'co_nlocals', 'co_posonlyargcount', 'co_stacksize', 'co_varnames', 'replace']


In [6]:
print(avg_closure1.__code__.co_freevars)

('series',)


In [7]:
print(dir(avg_closure1.__closure__))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'count', 'index']


In [10]:
# 잘못된 클로저 사용

def closure_ex2():
    # Free varialbe
    cnt = 0
    total = 0
    
    def averager(v):
        cnt += 1
        total += v
        return total / cnt
    
    return averager

avg_closure2 = closure_ex2()

print(avg_closure2(10))

UnboundLocalError: local variable 'cnt' referenced before assignment

In [ ]:
def closure_ex1():
    # Free variable
    # 클로저 영역
    series = []
    def averager(v):
        series.append(v)
        print('onner >>> {}/{}'.format(series, len(series)))
        return sum(series)/ len(series)
    return averager

이거랑 비교 해보자.

# 데코레이터
1. 장점
* 중복 제거, 코드 간결, 공통 함수 작성
* 로깅, 프레임워크, 유혀성 체크 -> 공통 기능
* 조합해서 사용 용이

2. 단점
* 가독성 감소
* 특정 기능에 한정된 함수 -> 단일 함수로 작성하는 것이 유리
* 디버깅 불편



In [14]:
import time

def perf_clock(func):
    def perf_clocked(*args):
        # 함수 시작 시간
        st = time.perf_counter()
        # 함수 실행
        result = func(*args)
        # 함수 종료 시간
        et = time.perf_counter() -st
        # 실행 함수명
        name = func.__name__
        # 함수 매개변수
        arg_str = ', '.join(repr(arg) for arg in args)
        # 결과 출력
        print('[%0.5fs] %s(%s) -> %r' % (et, name, arg_str, result))       
        return result
    return perf_clocked

@perf_clock
def time_func(seconds):
    time.sleep(seconds)

@perf_clock
def sum_func(*numbers):
    return sum(numbers)

# 데코레이터 미사용
none_deco1 = perf_clock(time_func)
none_deco2 = perf_clock(sum_func)

print(none_deco1, none_deco1.__code__.co_freevars)
print(none_deco2, none_deco2.__code__.co_freevars)

print('-' * 40, 'Called None Decorator -> time_func')
print()
none_deco1(1.5)
print('-' * 40, 'Called None Decorator -> sum_func')
print()
none_deco2(100, 150, 250, 300, 350)

print()
print()


# 데코레이터 사용
print('*' * 40, 'Called Decorator -> time_func')
print()
time_func(1.5)
print('*' * 40, 'Called Decorator -> sum_func')
print()
sum_func(100, 150, 250, 300, 350)
print()

<function perf_clock.<locals>.perf_clocked at 0x0000014D50D9B940> ('func',)
<function perf_clock.<locals>.perf_clocked at 0x0000014D50D9BA60> ('func',)
---------------------------------------- Called None Decorator -> time_func

[1.50459s] time_func(1.5) -> None
[1.50471s] perf_clocked(1.5) -> None
---------------------------------------- Called None Decorator -> sum_func

[0.00000s] sum_func(100, 150, 250, 300, 350) -> 1150
[0.00003s] perf_clocked(100, 150, 250, 300, 350) -> 1150


**************************************** Called Decorator -> time_func

[1.50771s] time_func(1.5) -> None
**************************************** Called Decorator -> sum_func

[0.00000s] sum_func(100, 150, 250, 300, 350) -> 1150

